In [69]:
import pandas as pd
import numpy as np
import re 
import sympy as sp 

In [70]:
from math import gcd, isclose
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

In [71]:
df = pd.read_csv(r'/content/Bias_data.csv')

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   link               1298 non-null   object 
 1   id                 1298 non-null   object 
 2   space_group_core   1276 non-null   float64
 3   space_group_shell  1259 non-null   float64
 4   core               1298 non-null   object 
 5   shell              1298 non-null   object 
 6   h_range_max_koe    1245 non-null   float64
 7   chemical formula   1298 non-null   object 
 8   Tb                 988 non-null    float64
 9   TN                 1109 non-null   float64
 10  Tc                 1048 non-null   float64
 11  c1                 1298 non-null   object 
 12  c2                 1298 non-null   object 
 13  x                  1234 non-null   float64
 14  y                  1212 non-null   float64
 15  z                  1209 non-null   float64
 16  shape              1203 

In [73]:
columns = ['link'] # del id
df = df.drop(columns = columns)
df

,id,space_group_core,space_group_shell,core,shell,h_range_max_koe,chemical formula,Tb,TN,Tc,...,shape,temperature_k,sat_em_g,coer_oe,mr (emu/g),exc_bias_oe,exc_dir,ver_shift_emu_g,ver_s_dir,fc_field_t
0,1,186.0,227.0,Zn1O1,Co1Fe2O4,50.0,ZnO-CoFe2O4,300.0,NaN,793.0,...,1.0,5.0,NaN,27800.000,NaN,1132.000,0.0,0.000,1.0,5.0
1,2,225.0,227.0,Co1O1,Co1Fe2O4,50.0,CoO-CoFe2O4,276.0,293.0,843.0,...,1.0,5.0,NaN,7800.000,NaN,406.000,0.0,0.000,1.0,5.0
2,3,227.0,0.0,Mn0.50Zn0.50Fe2O4,0,10.0,Mn0.5Zn0.5Fe2O4,150.0,NaN,640.0,...,1.0,300.0,2.290,24.800,0.0078,51.340,1.0,0.000,1.0,0.0
3,4,227.0,0.0,Mn0.50Zn0.50Fe2O4,0,10.0,Mn0.5Zn0.5Fe2O4,150.0,NaN,640.0,...,1.0,300.0,2.430,90.200,0.0218,108.180,1.0,0.000,1.0,0.0
4,5,227.0,0.0,Mn0.50Zn0.50Fe2O4,0,10.0,Mn0.5Zn0.5Fe2O4,150.0,NaN,640.0,...,1.0,300.0,16.150,40.070,0.3498,8.920,1.0,0.000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,val 28,161.0,0.0,BiFeO3,0,70.0,BiFeO3-SG,200.0,640.0,1103.0,...,1.0,2.0,NaN,1923.000,NaN,1180.000,0.0,NaN,1.0,7.0
1294,val 29,161.0,0.0,BiFeO3,0,70.0,BiFeO3-SG,200.0,640.0,1103.0,...,1.0,2.0,NaN,450.000,NaN,126.000,0.0,NaN,1.0,7.0
1295,val 30,15.0,225.0,NiCo2O4,NiO,30.0,NiCo2O4-NiO,75.0,490.0,NaN,...,1.0,10.0,0.845,619.815,0.1070,432.315,0.0,0.081,1.0,0.3
1296,val 31,15.0,225.0,NiCo2O4,NiO,30.0,NiCo2O4-NiO,75.0,490.0,NaN,...,1.0,30.0,0.783,289.735,0.6500,289.735,0.0,0.650,1.0,0.3


In [74]:
df['x'] = df['x'].astype('float64')
df['y'] = df['y'].astype('float64')
df['z'] = df['z'].astype('float64')

In [75]:
# Convert data types to numeric
df['shape'] = df['shape'].astype('float64')
df['space_group_core'] = df['space_group_core'].astype('float64')
df['space_group_shell'] = df['space_group_shell'].astype('float64')
# df['crystal_structure'] = df['crystal_structure'].astype('float64')

# Fill missing values in categorical features with the mode
df['shape'] = df['shape'].fillna(df['shape'].mode()[0])
df['space_group_core'] = df['space_group_core'].fillna(df['space_group_core'].mode()[0])
df['space_group_shell'] = df['space_group_shell'].fillna(df['space_group_shell'].mode()[0])
# df['crystal_structure'] = df['crystal_structure'].fillna(df['crystal_structure'].mode()[0])

# Convert data types to categorical
df['shape'] = df['shape'].astype('category')
df['space_group_core'] = df['space_group_core'].astype('category')
df['space_group_shell'] = df['space_group_shell'].astype('category')
# df['crystal_structure'] = df['crystal_structure'].astype('category')

In [76]:
df['exc_dir'] = df['exc_dir'].astype('category')
df['ver_s_dir'] = df['ver_s_dir'].astype('category')

In [77]:
df.columns

Index(['id', 'space_group_core', 'space_group_shell', 'core', 'shell',
       'h_range_max_koe', 'chemical formula', 'Tb', 'TN', 'Tc', 'c1', 'c2',
       'x', 'y', 'z', 'shape', 'temperature_k', 'sat_em_g', 'coer_oe',
       'mr (emu/g)', 'exc_bias_oe', 'exc_dir', 'ver_shift_emu_g', 'ver_s_dir',
       'fc_field_t'],
      dtype='object')

In [78]:
core, shel, all = df['c1'], df['c2'], df['chemical formula']

In [79]:
df  = df.drop(columns =['core', 'shell', 'chemical formula',
                        'c1', 'c2'])

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 1298 non-null   object  
 1   space_group_core   1298 non-null   category
 2   space_group_shell  1298 non-null   category
 3   h_range_max_koe    1245 non-null   float64 
 4   Tb                 988 non-null    float64 
 5   TN                 1109 non-null   float64 
 6   Tc                 1048 non-null   float64 
 7   x                  1234 non-null   float64 
 8   y                  1212 non-null   float64 
 9   z                  1209 non-null   float64 
 10  shape              1298 non-null   category
 11  temperature_k      1265 non-null   float64 
 12  sat_em_g           953 non-null    float64 
 13  coer_oe            1252 non-null   float64 
 14  mr (emu/g)         1024 non-null   float64 
 15  exc_bias_oe        1267 non-null   float64 
 16  exc_di

In [81]:
df_for_kNN = df.drop(columns=['id'])

# Normalize the data
scaler = StandardScaler()
normalized_data = scaler.fit_transform(df_for_kNN)

# Fill missing values in the dataset using kNN
imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(normalized_data)
df_imputed = pd.DataFrame(imputed_data, columns=df_for_kNN.columns)

# Combine the filled data with the original dataset
df_filled = pd.concat([df[['id']], df_imputed], axis=1)

# Print information about missing values after imputation
print(f'Percentage of missing values after imputation: \n{df_filled.isna().sum() * 100 / len(df_filled)}')

Percentage of missing values after imputation: 
id                   0.0
space_group_core     0.0
space_group_shell    0.0
h_range_max_koe      0.0
Tb                   0.0
TN                   0.0
Tc                   0.0
x                    0.0
y                    0.0
z                    0.0
shape                0.0
temperature_k        0.0
sat_em_g             0.0
coer_oe              0.0
mr (emu/g)           0.0
exc_bias_oe          0.0
exc_dir              0.0
ver_shift_emu_g      0.0
ver_s_dir            0.0
fc_field_t           0.0
dtype: float64


In [82]:
df_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1298 non-null   object 
 1   space_group_core   1298 non-null   float64
 2   space_group_shell  1298 non-null   float64
 3   h_range_max_koe    1298 non-null   float64
 4   Tb                 1298 non-null   float64
 5   TN                 1298 non-null   float64
 6   Tc                 1298 non-null   float64
 7   x                  1298 non-null   float64
 8   y                  1298 non-null   float64
 9   z                  1298 non-null   float64
 10  shape              1298 non-null   float64
 11  temperature_k      1298 non-null   float64
 12  sat_em_g           1298 non-null   float64
 13  coer_oe            1298 non-null   float64
 14  mr (emu/g)         1298 non-null   float64
 15  exc_bias_oe        1298 non-null   float64
 16  exc_dir            1298 

In [83]:
# Transform back to our original values
org_copy = scaler.inverse_transform(df_filled.iloc[:, 1:])  # Excluding the id column
df_recopy = pd.DataFrame(org_copy, columns=df_filled.columns[1:])  # Data in its original form

df_recopy['id'] = id
print(df_recopy['sat_em_g'].unique())

[51.8758 59.734   2.29   ...  0.845   0.783   0.67  ]


In [84]:
df = df_recopy

In [85]:
# Add columns for our values
df.insert(0, 'area', 0)
df.insert(0, 'volume', 0)
df = df.astype({'volume': float ,'area': float})

In [86]:
# Create dictionaries for calculating area and volume
dict_area = {
    1: lambda x, y=1, z=1: 4 * 3.14159 * x ** 2,  # Sphere
    2: lambda x, y=1, z=1: (3 ** 2) * 2 * x ** 2,  # Area for case 2
    3: lambda x, y=1, z=1: 6 * x ** 2,  # Cube
    4: lambda x, y=1, z=1: (3 ** (1/2)) * x ** 2,  # Regular tetrahedron
    5: lambda x, y=1, z=1: (3 ** (1/2)) * x ** 2,  # Repeated case 4
    6: lambda x, y=1, z=0: 2 * 3.14159 * (x / 2) * x + 2 * 3.14159 * (x / 2) ** 2,  # Cylinder
    7: lambda x, y=1, z=1: 6 * x ** 2,  # Cube (another case)
}

dict_volume = {
    1: lambda x, y=1, z=1: (4/3) * 3.14159 * x ** 3,  # Sphere
    2: lambda x, y=1, z=1: (2/3) * (2 ** 2) * x ** 3,  # Volume for case 2
    3: lambda x, y=1, z=1: x ** 3,  # Cube
    4: lambda x, y=1, z=1: (((2) ** (1/2)) * x ** 3) / 12,  # Regular tetrahedron
    5: lambda x, y=1, z=1: (((2) ** (1/2)) * x ** 3) / 12,  # Repeated case 4
    6: lambda x, y=1, z=0: 3.14159 * (x / 2) ** 2 * y,  # Cylinder
    7: lambda x, y=1, z=1: x ** 3  # Cube  another case)
}

In [87]:
# Create a function to calculate area
def calculate_area(row):
    shape = float(row['shape'])
    x = row['x']
    y = row.get('y', 0)
    z = row.get('z', 0)

    if shape in dict_area:
        if shape in [1, 2, 3, 4, 5, 6, 7]:
            return dict_area[shape](x, y) if shape in [3, 6] else dict_area[shape](x, z) if shape == 4 else dict_area[shape](x)
    else:
        return 'error'

# Create a function to calculate volume
def calculate_volume(row):
    shape = float(row['shape'])
    x = row['x']
    y = row.get('y', 0)
    z = row.get('z', 0)

    if shape in dict_volume:
        if shape in [1, 2, 3, 4, 5, 6, 7]:
            return dict_volume[shape](x, y, z) if shape in [3, 6] else dict_volume[shape](x, z) if shape == 4 else dict_volume[shape](x)
    else:
        return 'error'

In [88]:
df['area'] = df.apply(calculate_area, axis=1)
df['volume'] = df.apply(calculate_volume, axis=1)

In [89]:
print(df['area'].unique())

[6.88133874e+02 6.15751640e+02 3.63167804e+03 1.52052956e+03
 1.80955584e+03 4.07150064e+03 5.54176476e+03 9.16087644e+03
 1.62860026e+04 1.13097240e+04 2.49415316e+02 4.43405007e+02
 1.03680000e+04 1.96020000e+04 3.93691492e+03 5.22961638e+03
 7.42030992e+03 1.36019537e+04 7.85397500e+03 5.02654400e+05
 4.71238500e+06 5.30928710e+04 1.70553780e+03 4.63246295e+03
 5.57644791e+03 1.36847660e+04 3.40055805e+04 1.26676763e+04
 4.53645596e+03 2.12371484e+03 2.09116797e+03 2.82743100e+03
 3.54672945e+03 4.34745791e+03 5.02654400e+03 1.21500000e+04
 1.38240000e+04 1.56060000e+04 2.40000000e+03 3.21698816e+03
 3.75000000e+03 3.52800000e+03 6.14400000e+03 7.35000000e+03
 2.12176224e+03 2.39313760e+03 1.25663600e+03 8.04247040e+02
 5.83200000e+03 1.60459851e+03 4.02638741e+03 6.82215118e+03
 1.74736824e+03 1.74736824e+03 3.45194737e+03 8.60894024e+03
 2.01061760e+04 2.43284730e+04 3.01718304e+04 5.14718106e+04
 1.80123641e+04 2.32351996e+04 1.77952224e+03 1.38544119e+03
 1.20687321e+03 9.953813

In [90]:
def calculate_sphericity(row):
    area = row['area']
    volume = row['volume']

    # Check if area and volume are greater than zero
    if area <= 0 or volume <= 0:
        return float('nan')  # Return NaN for invalid data

    radius_sphere = (3 * volume / (4 * np.pi)) ** (1/3)
    surface_area_sphere = 4 * np.pi * radius_sphere ** 2
    sphericity = surface_area_sphere / area

    # Limit sphericity within the range of 0 to 1
    sphericity = max(0, min(1, sphericity))

    return sphericity

# Apply the function to the entire DataFrame
df['sphericity'] = df.apply(calculate_sphericity, axis=1)

In [91]:
print(df['sphericity'].unique())

[1.         0.67113929 0.67113929 0.51664327 0.06436598 0.40775738
 0.80599598 0.80599598 0.80599598 0.67113929 0.51664327 0.49325101
 0.53995481 0.51664327 0.67113929 0.23272239 0.92055376 0.9598049
 0.80599598 0.80599598 0.55615782 0.21839518 0.84278499 0.24326802
 0.87358071 0.13388663 0.26958259 0.03585747 0.05692019]


In [92]:
def compute_max_min_ratio(df):
    max_min_ratios = []
    for index, row in df.iterrows():
        values = [row['x'], row['y'], row['z']]
        non_zero_values = [value for value in values if value != 0]
        # Check if there are any non-zero values
        if non_zero_values:
            max_value = max(non_zero_values)
            min_value = min(non_zero_values)
            if min_value != 0:  # Ensure we don't divide by zero
                ratio = max_value / min_value
                max_min_ratios.append(ratio)
            else:
                max_min_ratios.append(0)
        else:
            max_min_ratios.append(0)

    df['max/min'] = max_min_ratios
    return df

# Apply the function
df = compute_max_min_ratio(df)

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   volume             1298 non-null   float64
 1   area               1298 non-null   float64
 2   space_group_core   1298 non-null   float64
 3   space_group_shell  1298 non-null   float64
 4   h_range_max_koe    1298 non-null   float64
 5   Tb                 1298 non-null   float64
 6   TN                 1298 non-null   float64
 7   Tc                 1298 non-null   float64
 8   x                  1298 non-null   float64
 9   y                  1298 non-null   float64
 10  z                  1298 non-null   float64
 11  shape              1298 non-null   float64
 12  temperature_k      1298 non-null   float64
 13  sat_em_g           1298 non-null   float64
 14  coer_oe            1298 non-null   float64
 15  mr (emu/g)         1298 non-null   float64
 16  exc_bias_oe        1298 

In [94]:
df.insert(0, 'area/volume', 0)
df = df.astype({'area/volume': float})

In [95]:
# Calculate the ratio of area to volume
df['area/volume'] = df['area'] / df['volume']

In [96]:
df['area/volume'].unique()

array([0.40540541, 0.42857143, 0.17647059, 0.27272727, 0.25      ,
       0.16666667, 0.14285714, 0.11111111, 0.08333333, 0.1       ,
       1.22474487, 0.91855865, 0.28125   , 0.20454545, 0.16949153,
       0.14705882, 0.12345679, 0.09118541, 0.12      , 0.015     ,
       0.3       , 0.04615385, 0.25751073, 0.15625   , 0.54694622,
       0.09090909, 0.05767013, 0.09448819, 0.15789474, 0.23076923,
       0.23255814, 0.2       , 0.17857143, 0.16129032, 0.15      ,
       0.13333333, 0.125     , 0.11764706, 0.1875    , 0.24      ,
       0.48214286, 0.17142857, 0.41991253, 0.2173913 , 0.3       ,
       0.375     , 0.26548673, 0.16759777, 0.12875536, 0.25440977,
       0.25440977, 0.1810064 , 0.11461756, 0.075     , 0.06818182,
       0.06122449, 0.046875  , 0.0792393 , 0.06976744, 0.25210084,
       0.28571429, 0.30612245, 0.03370787, 0.03333333, 0.44510386,
       0.32258065, 0.24193548, 0.23809524, 0.37974684, 0.35294118,
       0.75      , 0.33333333, 0.16224986, 0.05      , 0.29411

In [97]:
df['core'] = core
df['shell'] = shel
df['formula'] = all

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   area/volume        1298 non-null   float64
 1   volume             1298 non-null   float64
 2   area               1298 non-null   float64
 3   space_group_core   1298 non-null   float64
 4   space_group_shell  1298 non-null   float64
 5   h_range_max_koe    1298 non-null   float64
 6   Tb                 1298 non-null   float64
 7   TN                 1298 non-null   float64
 8   Tc                 1298 non-null   float64
 9   x                  1298 non-null   float64
 10  y                  1298 non-null   float64
 11  z                  1298 non-null   float64
 12  shape              1298 non-null   float64
 13  temperature_k      1298 non-null   float64
 14  sat_em_g           1298 non-null   float64
 15  coer_oe            1298 non-null   float64
 16  mr (emu/g)         1298 

In [99]:
# Export the data to a separate dataset
df.to_csv(r'second_bias.csv')